# Проверка кода агента

Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.


In [2]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY = load_mock_inputs()
print(f"Ключи data: {list(DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())[:5]}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [3]:
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig
from mas_automl.code_agent.base_pipeline import choose_framework

client = LLMClient(LLMConfig())
framework, reason = choose_framework(DATA_ANALYZE, METADATA, REGISTRY, client)
print(framework)
print(reason)


AutoGluon
Эвристика на основе типа задачи и размера выборки.


In [4]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


import pandas as pd
from autogluon.tabular import TabularPredictor

def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:
    predictor = TabularPredictor(label=label)
    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)
    predictor.evaluate(test_df, silent=True)
    return predictor
(True, 'Проверки пройдены.')


In [5]:
from mas_automl.code_agent.base_pipeline import run_pipeline

result = run_pipeline(max_iterations=2, llm=client)
result


PipelineResult(framework='AutoGluon', reason='Эвристика на основе типа задачи и размера выборки.', code="import pandas as pd\nfrom autogluon.tabular import TabularPredictor\n\ndef train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:\n    predictor = TabularPredictor(label=label)\n    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)\n    predictor.evaluate(test_df, silent=True)\n    return predictor", tests_passed=True, iterations=1, feedback='Проверки пройдены.')